# Pandas Best Practices Demonstration

This Jupyter Notebook, `Pandas_1.ipynb`, is designed to showcase some of the best practices in pandas. Throughout this notebook, we will explore various pandas techniques that aim is to provide a practical guide to writing clean, memory efficient, and maintainable Python code. 

Let's dive in and start exploring Pandas best practices!

## Summary
- use PyArrow
- review the data types after loading the dataframe
- get function docs in place if possible. 
- check memory usage
- chain your transformations
- split the transformations maps/dictionaries 
- use `.query()`
- define your filtering contions as variables.

## Introduction

* Python can be very slow when you don't use the right tools and data types specially when you handle datasets because in Python "everything is an object"

* There are continuous efforts to increase the scalability and the speed of pandas operations: Like Modin, `modin.pandas` data tool that implements Pandas API  to speed up the data loading and `apply` function 

* PyArrow is introduced as an API to provide Arrow C++ functionality and interoperability with Pandas and Numpy 


In [1]:
%load_ext memory_profiler

In [2]:
import pandas as pd
# import modin.pandas as pd
import numpy as np
import pyarrow as pa

In [3]:
pd.__version__, np.__version__, pa.__version__

('2.2.1', '1.26.4', '15.0.2')

## Loading data

* Pandas enables choosing an engine to parse the loaded data in the dataframe. The default engine is Numpy, but we can also use Pyarrow, which is faster and more memory efficient.

#### We will time our code and check the memory usage as we go

In [4]:
%%time
df_np = pd.read_csv('data/GSS.csv')

CPU times: total: 219 ms
Wall time: 222 ms


In [5]:
%%time
# using PyArrow
df_ar = pd.read_csv('data/GSS.csv', dtype_backend='pyarrow', engine='pyarrow')

CPU times: total: 125 ms
Wall time: 96.3 ms


In [6]:
%%memit
gss_np = pd.read_csv('data/GSS.csv', index_col=0)

peak memory: 166.24 MiB, increment: 7.55 MiB


In [7]:
%%memit
# using PyArrow
df_ar = pd.read_csv('data/GSS.csv', dtype_backend='pyarrow', engine='pyarrow')

peak memory: 181.15 MiB, increment: 15.68 MiB


## Why PyArrow?

- PyaArrow enables faster conversion of dataframes between packages like pandas and polars(build using Rust Arrow ) as blob 

- PyArrow native string types saves memory over default pandas one.

- PyArrow doesn't cast columns with integers + missing values to float columns like Numpy.
- PyArrow will become a required dependency with pandas 3.0 [docs](https://pandas.pydata.org/docs/whatsnew/v2.1.0.html#pyarrow-will-become-a-required-dependency-with-pandas-3-0)

In [18]:
%%time
gss_np = pd.read_csv('data/GSS.csv', index_col=0)
gss_np.memory_usage(deep=True).sum() # ~35 MB

CPU times: total: 281 ms
Wall time: 293 ms


36076324

In [9]:
gss_np.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64814 entries, 0 to 64813
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   YEAR      64814 non-null  int64  
 1   ID        64814 non-null  int64  
 2   AGE       64586 non-null  float64
 3   HRS1      37506 non-null  float64
 4   OCC       64814 non-null  int64  
 5   MAJOR1    64814 non-null  object 
 6   SEX       64814 non-null  object 
 7   RACE      64814 non-null  object 
 8   BORN      64814 non-null  object 
 9   INCOME    64814 non-null  object 
 10  INCOME06  64814 non-null  object 
 11  HONEST    64814 non-null  object 
 12  TICKET    64814 non-null  object 
dtypes: float64(2), int64(3), object(8)
memory usage: 6.9+ MB


- To get column by datatype use `select_dtypes` and pass parmeters for type: 
    - spesific type like `int8`
    - generic like `integer`
    - or `np.number`
- To get more details you can  check the docs using command `df.select_dtypes?`

In [12]:
gss_np.select_dtypes?

Signature: gss_np.select_dtypes(include=None, exclude=None) -> 'Self'
Docstring:
Return a subset of the DataFrame's columns based on the column dtypes.

Parameters
----------
include, exclude : scalar or list-like
    A selection of dtypes or strings to be included/excluded. At least
    one of these parameters must be supplied.

Returns
-------
DataFrame
    The subset of the frame including the dtypes in ``include`` and
    excluding the dtypes in ``exclude``.

Raises
------
ValueError
    * If both of ``include`` and ``exclude`` are empty
    * If ``include`` and ``exclude`` have overlapping elements
    * If any kind of string dtype is passed in.

See Also
--------
DataFrame.dtypes: Return Series with the data type of each column.

Notes
-----
* To select all *numeric* types, use ``np.number`` or ``'number'``
* To select strings you must use the ``object`` dtype, but note that
  this will return *all* object dtype columns
* See the `numpy dtype hierarchy
  <https://numpy.org/doc/st

In [13]:
%%time
gss = pd.read_csv('data/GSS.csv', index_col=0, dtype_backend='pyarrow', engine='pyarrow')

CPU times: total: 62.5 ms
Wall time: 32.6 ms


In [20]:
gss.memory_usage(deep=True).sum() # ~8.5 MB

8611400

In [11]:
gss.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64814 entries, 0 to 64813
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype          
---  ------    --------------  -----          
 0   YEAR      64814 non-null  int64[pyarrow] 
 1   ID        64814 non-null  int64[pyarrow] 
 2   AGE       64586 non-null  double[pyarrow]
 3   HRS1      37506 non-null  double[pyarrow]
 4   OCC       64814 non-null  int64[pyarrow] 
 5   MAJOR1    64814 non-null  string[pyarrow]
 6   SEX       64814 non-null  string[pyarrow]
 7   RACE      64814 non-null  string[pyarrow]
 8   BORN      64814 non-null  string[pyarrow]
 9   INCOME    64814 non-null  string[pyarrow]
 10  INCOME06  64814 non-null  string[pyarrow]
 11  HONEST    64814 non-null  string[pyarrow]
 12  TICKET    64814 non-null  string[pyarrow]
dtypes: double[pyarrow](2), int64[pyarrow](3), string[pyarrow](8)
memory usage: 8.2 MB


In [21]:
# numpy has int data types if you need to get details and limit : 
import numpy as np
np.iinfo(np.int8), np.iinfo(np.int16), np.iinfo(np.int32), np.iinfo(np.int64)

(iinfo(min=-128, max=127, dtype=int8),
 iinfo(min=-32768, max=32767, dtype=int16),
 iinfo(min=-2147483648, max=2147483647, dtype=int32),
 iinfo(min=-9223372036854775808, max=9223372036854775807, dtype=int64))

#### Pandas Practice

- chaining 
it makes the code more readable as set of steps or a receipe with one line at a time 

In [22]:

# nlargest and nsmallest vs sort_values
gss['AGE'].nlargest(3)



1179    89.0
1228    89.0
2247    89.0
Name: AGE, dtype: double[pyarrow]

In [23]:
gss['AGE'].nsmallest(3)


419     18.0
881     18.0
1100    18.0
Name: AGE, dtype: double[pyarrow]

In [24]:
# chaining 
(gss['AGE']
 .sort_values(ascending=False)
 .head(3)
 )


1179    89.0
1228    89.0
2247    89.0
Name: AGE, dtype: double[pyarrow]

In [25]:
#%%time
(gss
 .nlargest(3, 'AGE', keep='all') 
 )

,YEAR,ID,AGE,HRS1,OCC,MAJOR1,SEX,RACE,BORN,INCOME,INCOME06,HONEST,TICKET
,,,,,,,,,,,,,
1179,1972,1180,89.0,<NA>,915,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
1228,1972,1229,89.0,<NA>,761,No answer,Male,White,No answer,No answer,Refused,No answer,No answer
2247,1973,635,89.0,<NA>,551,No answer,Male,White,No answer,3000,Refused,3 most desireable,No
3127,1974,11,89.0,<NA>,265,No answer,Male,Black,No answer,4000,Refused,No answer,No
3379,1974,263,89.0,<NA>,9999,No answer,Female,White,No answer,1000,Refused,No answer,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64263,2018,1798,89.0,<NA>,9999,Psychology,Female,White,Yes,No answer,Refused,No answer,No answer
64544,2018,2079,89.0,<NA>,9999,No answer,Male,White,Yes,No answer,Refused,No answer,No answer
64610,2018,2145,89.0,60.0,9999,No answer,Male,White,Yes,25000,Refused,No answer,No answer


In [22]:
#%%time
(gss
 .nsmallest(3, 'AGE', keep='all') 
 )
# keep='all' to show all rows with the same value

,YEAR,ID,AGE,HRS1,OCC,MAJOR1,SEX,RACE,BORN,INCOME,INCOME06,HONEST,TICKET
,,,,,,,,,,,,,
419,1972,420,18.0,<NA>,9999,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
881,1972,882,18.0,<NA>,9999,No answer,Male,White,No answer,No answer,Refused,No answer,No answer
1100,1972,1101,18.0,<NA>,623,No answer,Male,White,No answer,No answer,Refused,No answer,No answer
1547,1972,1548,18.0,<NA>,9999,No answer,Female,Black,No answer,No answer,Refused,No answer,No answer
1578,1972,1579,18.0,<NA>,9999,No answer,Male,Black,No answer,No answer,Refused,No answer,No answer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64303,2018,1838,18.0,<NA>,9999,No answer,Female,White,Yes,No answer,Refused,No answer,No answer
64343,2018,1878,18.0,<NA>,9999,No answer,Male,White,Yes,25000,Refused,No answer,No answer
64407,2018,1942,18.0,<NA>,9999,No answer,Female,White,Yes,No answer,Refused,No answer,No answer


### Hints on transformation : 

#### Int types

- pandas using numpy will not throw intger overflow error instead the dataframe cell will have inaccurate negative values.

`soon we will see that ` 

In [23]:
# remeber this line 
# gss.select_dtypes(int).describ()

# is equal to 
(
    gss
    .select_dtypes(int)
    .describe()
)


,YEAR,ID,OCC
count,64814.0,64814.0,64814.0
mean,1994.93918,1151.810211,6418.583284
std,13.465368,828.030233,4618.278478
min,1972.0,1.0,1.0
25%,1984.0,507.0,613.0
50%,1996.0,1029.5,9999.0
75%,2006.0,1570.0,9999.0
max,2018.0,4510.0,9999.0


In [26]:
# chaining casting into pyarrow types
type_map = {'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]' }
(gss
 .astype(type_map)
 .select_dtypes(['uint16'])
 .describe()
)

,YEAR,ID,OCC
count,64814.0,64814.0,64814.0
mean,1994.93918,1151.810211,6418.583284
std,13.465368,828.030233,4618.278478
min,1972.0,1.0,1.0
25%,1984.0,507.0,613.0
50%,1996.0,1029.5,9999.0
75%,2006.0,1570.0,9999.0
max,2018.0,4510.0,9999.0


In [25]:
# chaining
# Error in casting year to uint8
# use 'integer' so see all int-like columns
type_map_2 = {'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]' }
(gss
 .astype(type_map_2) 
 .select_dtypes(['integer'])  
 .describe()
)

,YEAR,ID,OCC
count,64814.0,64814.0,64814.0
mean,1994.93918,1151.810211,6418.583284
std,13.465368,828.030233,4618.278478
min,1972.0,1.0,1.0
25%,1984.0,507.0,613.0
50%,1996.0,1029.5,9999.0
75%,2006.0,1570.0,9999.0
max,2018.0,4510.0,9999.0


In [26]:
# numpy
(gss
 .astype({'YEAR': 'int8'})
 .describe()
)

,YEAR,ID,AGE,HRS1,OCC
count,64814.000000,64814.0,64586.0,37506.0,64814.0
mean,-53.060820,1151.810211,46.099356,41.303711,6418.583284
std,13.465368,828.030233,17.534703,14.171808,4618.278478
min,-76.000000,1.0,18.0,0.0,1.0
25%,-64.000000,507.0,31.0,37.0,613.0
50%,-52.000000,1029.5,44.0,40.0,9999.0
75%,-42.000000,1570.0,59.0,48.0,9999.0
max,-30.000000,4510.0,89.0,89.0,9999.0


In [27]:
# pyarrow
(gss
 .astype({'YEAR': 'int8[pyarrow]'})
 .describe()
)

ArrowInvalid: Integer value 1972 not in range: -128 to 127: Error while type casting for column 'YEAR'

In [27]:
(gss_np
.select_dtypes('float'))

,AGE,HRS1
0,23.0,NaN
1,70.0,NaN
2,48.0,NaN
3,27.0,NaN
4,61.0,NaN
...,...,...
64809,37.0,36.0
64810,75.0,36.0
64811,67.0,NaN
64812,72.0,NaN


In [28]:
(gss_np
 .memory_usage(deep=True)
 .sum()
 )

36076324

In [29]:
# cast HRS1 to pyarrow int
casting_types = {'HRS1': 'int8[pyarrow]','AGE': 'int8[pyarrow]'}
(gss
 .astype(casting_types)
 .select_dtypes('integer')
 .describe()
)

,YEAR,ID,AGE,HRS1,OCC
count,64814.0,64814.0,64586.0,37506.0,64814.0
mean,1994.93918,1151.810211,46.099356,41.303711,6418.583284
std,13.465368,828.030233,17.534703,14.171808,4618.278478
min,1972.0,1.0,18.0,0.0,1.0
25%,1984.0,507.0,31.0,37.0,613.0
50%,1996.0,1029.5,44.0,40.0,9999.0
75%,2006.0,1570.0,59.0,48.0,9999.0
max,2018.0,4510.0,89.0,89.0,9999.0


In [30]:
casting_types = {'HRS1': 'int8[pyarrow]','AGE': 'int8[pyarrow]'}
(gss
 .astype(casting_types)
 .memory_usage(deep=True)
 .sum()
)

9817572

#### Finding values and NAs

query(`string`) is more readable and easier for chaining . 

In [31]:
(gss
  .query('AGE < 20')
)

,YEAR,ID,AGE,HRS1,OCC,MAJOR1,SEX,RACE,BORN,INCOME,INCOME06,HONEST,TICKET
,,,,,,,,,,,,,
74,1972,75,19.0,<NA>,753,No answer,Male,White,No answer,No answer,Refused,No answer,No answer
172,1972,173,19.0,<NA>,405,No answer,Male,White,No answer,No answer,Refused,No answer,No answer
177,1972,178,19.0,<NA>,422,No answer,Male,White,No answer,No answer,Refused,No answer,No answer
253,1972,254,19.0,<NA>,912,No answer,Male,White,No answer,No answer,Refused,No answer,No answer
340,1972,341,19.0,<NA>,785,No answer,Male,Black,No answer,No answer,Refused,No answer,No answer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64476,2018,2011,19.0,<NA>,9999,No answer,Male,White,Yes,0,Refused,No answer,No answer
64520,2018,2055,18.0,30.0,9999,No answer,Male,White,Yes,25000,Refused,No answer,No answer
64537,2018,2072,18.0,<NA>,9999,No answer,Female,White,No,No answer,Refused,No answer,No answer


In [33]:
# where are the missing values? 
# let's query

(gss
  .query('HRS1.isna()')
)


,YEAR,ID,AGE,HRS1,OCC,MAJOR1,SEX,RACE,BORN,INCOME,INCOME06,HONEST,TICKET
,,,,,,,,,,,,,
0,1972,1,23.0,<NA>,205,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
1,1972,2,70.0,<NA>,441,No answer,Male,White,No answer,No answer,Refused,No answer,No answer
2,1972,3,48.0,<NA>,270,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
3,1972,4,27.0,<NA>,1,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
4,1972,5,61.0,<NA>,385,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64807,2018,2342,68.0,<NA>,9999,No answer,Female,White,Yes,8000,Refused,No answer,No answer
64808,2018,2343,19.0,<NA>,9999,No answer,Male,White,Yes,15000,Refused,No answer,No answer
64811,2018,2346,67.0,<NA>,9999,No answer,Female,White,Yes,25000,Refused,No answer,No answer


In [32]:
(gss
  .query('AGE.isna()')
)

,YEAR,ID,AGE,HRS1,OCC,MAJOR1,SEX,RACE,BORN,INCOME,INCOME06,HONEST,TICKET
,,,,,,,,,,,,,
228,1972,229,<NA>,<NA>,280,No answer,Male,White,No answer,No answer,Refused,No answer,No answer
344,1972,345,<NA>,<NA>,9999,No answer,Female,Black,No answer,No answer,Refused,No answer,No answer
1038,1972,1039,<NA>,<NA>,370,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
1427,1972,1428,<NA>,<NA>,902,No answer,Male,Black,No answer,No answer,Refused,No answer,No answer
1553,1972,1554,<NA>,<NA>,395,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63124,2018,659,<NA>,<NA>,9999,No answer,Female,Black,Yes,25000,Refused,No answer,No answer
63906,2018,1441,<NA>,16.0,9999,No answer,Female,White,Yes,20000,Refused,No answer,No answer
63938,2018,1473,<NA>,40.0,9999,Business administration,Male,Black,Yes,25000,Refused,No answer,No answer


In [37]:
# let's see the missing values using different method
gss[gss['HRS1'].isna()]
#gss[gss['AGE'].isna()]

,YEAR,ID,AGE,HRS1,OCC,MAJOR1,SEX,RACE,BORN,INCOME,INCOME06,HONEST,TICKET
,,,,,,,,,,,,,
0,1972,1,23.0,<NA>,205,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
1,1972,2,70.0,<NA>,441,No answer,Male,White,No answer,No answer,Refused,No answer,No answer
2,1972,3,48.0,<NA>,270,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
3,1972,4,27.0,<NA>,1,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
4,1972,5,61.0,<NA>,385,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64807,2018,2342,68.0,<NA>,9999,No answer,Female,White,Yes,8000,Refused,No answer,No answer
64808,2018,2343,19.0,<NA>,9999,No answer,Male,White,Yes,15000,Refused,No answer,No answer
64811,2018,2346,67.0,<NA>,9999,No answer,Female,White,Yes,25000,Refused,No answer,No answer


In [33]:
# if the using [] is prefered over query 
# use python best practice for the condition 

NA_HR_filter = gss['HRS1'].isna()

gss[NA_HR_filter]

,YEAR,ID,AGE,HRS1,OCC,MAJOR1,SEX,RACE,BORN,INCOME,INCOME06,HONEST,TICKET
,,,,,,,,,,,,,
0,1972,1,23.0,<NA>,205,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
1,1972,2,70.0,<NA>,441,No answer,Male,White,No answer,No answer,Refused,No answer,No answer
2,1972,3,48.0,<NA>,270,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
3,1972,4,27.0,<NA>,1,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
4,1972,5,61.0,<NA>,385,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64807,2018,2342,68.0,<NA>,9999,No answer,Female,White,Yes,8000,Refused,No answer,No answer
64808,2018,2343,19.0,<NA>,9999,No answer,Male,White,Yes,15000,Refused,No answer,No answer
64811,2018,2346,67.0,<NA>,9999,No answer,Female,White,Yes,25000,Refused,No answer,No answer


In [36]:
# for adding more than one condition 
NA_HR_filter = gss['HRS1'].isna()
NA_AGE_filter = gss['AGE'].isna()

NA_AGE_HRS_filter = NA_HR_filter & NA_AGE_filter

gss[NA_AGE_HRS_filter]

,YEAR,ID,AGE,HRS1,OCC,MAJOR1,SEX,RACE,BORN,INCOME,INCOME06,HONEST,TICKET
,,,,,,,,,,,,,
228,1972,229,<NA>,<NA>,280,No answer,Male,White,No answer,No answer,Refused,No answer,No answer
344,1972,345,<NA>,<NA>,9999,No answer,Female,Black,No answer,No answer,Refused,No answer,No answer
1038,1972,1039,<NA>,<NA>,370,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
1427,1972,1428,<NA>,<NA>,902,No answer,Male,Black,No answer,No answer,Refused,No answer,No answer
1553,1972,1554,<NA>,<NA>,395,No answer,Female,White,No answer,No answer,Refused,No answer,No answer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58592,2014,1532,<NA>,<NA>,9999,No answer,Female,White,Yes,No answer,Refused,No answer,No answer
59165,2014,2110,<NA>,<NA>,9999,Political science/international relations,Female,White,Yes,No answer,Refused,No answer,No answer
60336,2016,738,<NA>,<NA>,9999,Business administration,Female,Black,Yes,25000,Refused,No answer,No answer
